In [12]:
import json
import pandas as pd
import re

In [46]:
split = 'train'


# Load the data
df = pd.read_csv(f'{split}.csv')

# convert to jsonl
data = []
for index, row in df.iterrows():
    sample = {}
    sample['id'] = row['id']
    sample['citing_paper'] = {
        'title': row['citing_title'] if row['citing_title'] != 'unknown' else None,
        'year': row['citing_year'] if row['citing_year'] != 'unknown' else None,
        'authors': str(row['citing_authors']).split(';') if row['citing_authors'] != 'unknown' else None
    }

    paragraph = eval(str(row['paragraph']))
    ref_loc = []
    for word_id, word in enumerate(paragraph):
        match = re.match(r'<ref[^>]*?>(.*?)</ref>', word)
        if not match: continue
        paragraph[word_id] = match.group(1)
        ref_loc.append(word_id)
    sample['text'] = paragraph

    # Parse the cited data into lists
    cited_titles = eval(str(row['cited_title']))
    cited_years = eval(str(row['cited_year']))
    cited_authors = [authors.split(';') for authors in eval(str(row['cited_authors']))]

    # Create list of cited paper dictionaries
    sample['cited_papers'] = []
    for i in range(len(cited_titles)):
        cited_paper = {
            'title': cited_titles[i] if cited_titles[i] != 'unknown' else None,
            'year': cited_years[i] if cited_years[i] != 'unknown' else None,
            'authors': cited_authors[i] if cited_authors[i] != 'unknown' else None,
        }
        sample['cited_papers'].append(cited_paper)

    sample['target_citation_location'] = int(row['target_reference_location'])
    sample['citation_locations'] = ref_loc
    sample['citation_type'] = row['citation_type']
    sample['annotations'] = [
        eval(str(row['context_location_1'])),
        # eval(str(row['context_location_2']))
    ]
    data.append(sample)


In [47]:
# save to jsonl using split
with open(f'{split}.jsonl', 'w') as f:
    for sample in data:
        f.write(json.dumps(sample) + '\n')